# Semantic Search Function 

#### imports

In [2]:
import polars as pl 
from sentence_transformers import SentenceTransformer
import numpy as np 
from sklearn.metrics import DistanceMetric
import gradio as gr 

#### load data, model and metric

In [ ]:
%time df = pl.scan_parquet('data/video-transcripts.parquet')


CPU times: user 1.13 ms, sys: 3.56 ms, total: 4.68 ms
Wall time: 8.25 ms


In [5]:
model_name = 'all-MiniLM-L6-v2'
%time model = SentenceTransformer(model_name)

CPU times: user 83.7 ms, sys: 93.1 ms, total: 177 ms
Wall time: 1.59 s


In [7]:
dist_name = 'manhattan'
%time dist = DistanceMetric.get_metric(dist_name)

CPU times: user 37 μs, sys: 76 μs, total: 113 μs
Wall time: 115 μs


#### Search Function 

In [10]:
def returnSearchResult(query: str, index: pl.LazyFrame) -> np.ndarray:
    '''
        Funtion to return the indexes of top search results
    '''
    query_embedding = model.encode(query)

    # Compute the distance between the query and titles/transcript 
    dist_arr = dist.pairwise(df.select(df.columns[4:388]).collect(), query_embedding) +  dist.pairwise(df.select(df.columns[388:]).collect(), query_embedding)

    # Search parameters 
    threshold = 40
    top_k = 5

    # evaluate videos close to query based on threshold
    idx_below_threshold = np.argwhere(dist_arr.flatten()<threshold).flatten()
    # keep top k closest videos
    idx_sorted = np.argsort(dist_arr[idx_below_threshold], axis=0).flatten()

    # return indexes of search results
    return idx_below_threshold[idx_sorted][:top_k]

In [ ]:
query = 'LLM'
idx_result = returnSearchResult(query, df)
print(df.select(['video_id', 'title']).collect()[idx_result])

/var/folders/vp/2hwjkxh92qsdfnjr9y_xpmm00000gn/T/ipykernel_1804/1268863700.py:8: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  dist_arr = dist.pairwise(df.select(df.columns[4:388]).collect(), query_embedding) +  dist.pairwise(df.select(df.columns[388:]).collect(), query_embedding)


ValueError: need at least one array to concatenate